In [60]:
import numpy as np
import pandas as pd
import sqlite3

In [61]:
def get_host_from_headers(x):
    temp = x.replace('"','').split('],[')
    for t in temp:
            pair = t.replace(']]','').replace('[[','').split(',')
            if "Host" in pair:
                host_value = pair[1]
    return host_value

def get_cookies_from_headers(x):
    ls = x.replace('"','').split('],[')
    for l in ls:
        ta = l.replace(']]','').split(',')
        if "Cookie" in ta:
            return ta[-1]
    return ''

In [82]:
for exp_number in range(1,2):
    exp_path = "./raw-data/exp"+str(exp_number)+"/crawl-data.sqlite"
    
    conn = sqlite3.connect(exp_path)
    javascript_cookies = pd.read_sql_query("select visit_id, record_type, change_cause, host, name, value, time_stamp from javascript_cookies;", conn)
    # http_responses = pd.read_sql_query("select visit_id, url, method, headers, time_stamp from http_responses where response_status == 200;", conn)
    http_requests = pd.read_sql_query("select visit_id, headers, referrer, Url, time_stamp from http_requests;", conn)
    javascript = pd.read_sql_query("select visit_id, script_url, top_level_url, symbol, operation, value, arguments, time_stamp from javascript;", conn)
    site_visits = pd.read_sql_query("select visit_id, site_url from site_visits;", conn)
    
    conn.close()
    
    names = list(map(lambda x: x.split('.')[1], list(site_visits.site_url)))
    websites = list( names[i] for i in [0,5,10])
    
    web_dict = {
      "nytimes": 'nytimes|nyt',
      "webmd": 'webmd',
      "berkeley": 'berkeley'
    }
    
    

In [83]:
javascript

,visit_id,script_url,top_level_url,symbol,operation,value,arguments,time_stamp
0,1,https://www.nytimes.com/,https://www.nytimes.com/,window.navigator.userAgent,get,Mozilla/5.0 (X11; Linux x86_64; rv:72.0) Gecko...,None,2020-04-28T14:10:30.230Z
1,1,https://www.nytimes.com/,https://www.nytimes.com/,window.navigator.userAgent,get,Mozilla/5.0 (X11; Linux x86_64; rv:72.0) Gecko...,None,2020-04-28T14:10:30.262Z
2,1,https://www.nytimes.com/,https://www.nytimes.com/,window.navigator.userAgent,get,Mozilla/5.0 (X11; Linux x86_64; rv:72.0) Gecko...,None,2020-04-28T14:10:30.265Z
3,1,https://www.nytimes.com/,https://www.nytimes.com/,window.document.referrer,get,,None,2020-04-28T14:10:30.265Z
4,1,https://www.nytimes.com/,https://www.nytimes.com/,window.document.cookie,get,nyt-gdpr=0; nyt-geo=US; nyt-a=4Oe53a4sDRVl1-qj...,None,2020-04-28T14:10:30.265Z
5,1,https://www.nytimes.com/,https://www.nytimes.com/,window.document.cookie,get,nyt-gdpr=0; nyt-geo=US; nyt-a=4Oe53a4sDRVl1-qj...,None,2020-04-28T14:10:30.267Z
6,1,https://www.nytimes.com/,https://www.nytimes.com/,window.document.cookie,get,nyt-gdpr=0; nyt-geo=US; nyt-a=4Oe53a4sDRVl1-qj...,None,2020-04-28T14:10:30.267Z
7,1,https://www.nytimes.com/,https://www.nytimes.com/,window.document.cookie,get,nyt-gdpr=0; nyt-geo=US; nyt-a=4Oe53a4sDRVl1-qj...,None,2020-04-28T14:10:30.269Z
8,1,https://www.nytimes.com/,https://www.nytimes.com/,window.document.cookie,get,nyt-gdpr=0; nyt-geo=US; nyt-a=4Oe53a4sDRVl1-qj...,None,2020-04-28T14:10:30.270Z
9,1,https://www.nytimes.com/,https://www.nytimes.com/,window.document.cookie,get,nyt-gdpr=0; nyt-geo=US; nyt-a=4Oe53a4sDRVl1-qj...,None,2020-04-28T14:10:30.270Z


In [85]:
indx1 = findIdx(javascript_cookies, 'host', websites[1])
indx2 = findIdx(javascript_cookies, 'host', websites[2])
javascript_cookies["context_id"] = [1] * (indx1)  + [2] * (indx2 - indx1) + [3] * (javascript_cookies.shape[0] - indx2)

indx1 = findIdx(http_requests, 'url', websites[1])
indx2 = findIdx(http_requests,'url', websites[2])
http_requests["context_id"] = [1] * (indx1)  + [2] * (indx2 - indx1) + [3] * (http_requests.shape[0] - indx2)

indx1 = findIdx(javascript, 'top_level_url', websites[1])
indx2 = findIdx(javascript, 'top_level_url', websites[2])
javascript["context_id"] = [1] * (indx1)  + [2] * (indx2 - indx1) + [3] * (javascript.shape[0] - indx2)

In [84]:
def findIdx(df, colname, keyword):
    for i, web in enumerate(df[colname].tolist()):
        if web.rfind(keyword) != -1:
            return i

In [86]:
for context_number in range(1,4):
        
        ############################# cookies #############################
        js_cookies = javascript_cookies[javascript_cookies.context_id == context_number]
    
        # first party cookies (filter by host)
        js_cookies_first = js_cookies[js_cookies.host.str.contains(web_dict.get(websites[context_number-1]))]

        # third party cookies (filter by host)
        js_cookies_third = js_cookies[~js_cookies.host.str.contains(web_dict.get(websites[context_number-1]))]

        js_cookies_third.to_csv(str(exp_number)+'-'+str(context_number)+'-ThirdCookies.csv', index=False)
        
        
        ############################# JS #############################
        js = javascript[javascript.context_id == context_number]
        
        # filter by script_url (get domain from script_url)
        temp = list(map(lambda x:x[1], js['script_url'].str.split(".")))
        js['temp'] = temp
        
        js_third = js[~js.script_url.str.contains(web_dict.get(websites[context_number-1]))]

        js_third.to_csv(str(exp_number)+'-'+str(context_number)+'-js.csv', index=False)
        
        
        ############################# HTTP REQUEST #############################
        temp = list(map(lambda x: get_host_from_headers(x), http_requests.headers.tolist()))
        http_requests["host"] = temp

        http_req_cookies = list(map(lambda x: get_cookies_from_headers(x), http_requests.headers.tolist()))
        http_requests["cookies"] = http_req_cookies

        # get domain from host (eg. "ade.googlesyndication.com" -> "googlesyndication")
        hr = http_requests[http_requests.context_id == context_number]

        temp = list(map(lambda x:x[-2], hr['host'].str.split(".")))
        hr['temp'] = temp
        
        hr_third = hr[~hr.temp.str.contains(web_dict.get(websites[context_number-1]))]

        hr_third.to_csv(str(exp_number)+'-'+str(context_number)+'-httprequest.csv', index=False)



/Users/rainyleon/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/rainyleon/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [87]:
js

,visit_id,script_url,top_level_url,symbol,operation,value,arguments,time_stamp,context_id,temp
25242,11,https://use.typekit.net/zpy2xov.js,https://www.berkeley.edu/,window.navigator.userAgent,get,Mozilla/5.0 (X11; Linux x86_64; rv:72.0) Gecko...,None,2020-04-28T14:15:45.892Z,3,typekit
25243,11,https://use.typekit.net/zpy2xov.js,https://www.berkeley.edu/,window.navigator.userAgent,get,Mozilla/5.0 (X11; Linux x86_64; rv:72.0) Gecko...,None,2020-04-28T14:15:46.985Z,3,typekit
25244,11,https://use.typekit.net/zpy2xov.js,https://www.berkeley.edu/,window.navigator.userAgent,get,Mozilla/5.0 (X11; Linux x86_64; rv:72.0) Gecko...,None,2020-04-28T14:15:46.986Z,3,typekit
25245,11,https://siteimproveanalytics.com/js/siteanalyz...,https://www.berkeley.edu/,window.document.referrer,get,,None,2020-04-28T14:15:46.986Z,3,com/js/siteanalyze_8343
25246,11,https://siteimproveanalytics.com/js/siteanalyz...,https://www.berkeley.edu/,window.document.referrer,get,,None,2020-04-28T14:15:46.988Z,3,com/js/siteanalyze_8343
25247,11,https://ssl.google-analytics.com/ga.js,https://www.berkeley.edu/,window.document.referrer,get,,None,2020-04-28T14:15:46.990Z,3,google-analytics
25248,11,https://ssl.google-analytics.com/ga.js,https://www.berkeley.edu/,window.document.cookie,get,,None,2020-04-28T14:15:46.991Z,3,google-analytics
25249,11,https://ssl.google-analytics.com/ga.js,https://www.berkeley.edu/,window.screen.colorDepth,get,24,None,2020-04-28T14:15:46.993Z,3,google-analytics
25250,11,https://ssl.google-analytics.com/ga.js,https://www.berkeley.edu/,window.navigator.language,get,en-US,None,2020-04-28T14:15:46.993Z,3,google-analytics
25251,11,https://ssl.google-analytics.com/ga.js,https://www.berkeley.edu/,window.navigator.appName,get,Netscape,None,2020-04-28T14:15:46.995Z,3,google-analytics
